In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

In [2]:
class Conv_block(nn.Module):
    def __init__(self,in_channel,out_channel,**kwargs):
   s     super(Conv_block,self).__init__()
        
        self.network=nn.Sequential(
            nn.Conv2d(in_channel,out_channel,**kwargs),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )
    def forward(self,x):
        return self.network(x)

In [3]:
class Inception_block(nn.Module):
    def __init__(
        self,in_channel,out_1x1,red_3x3,out_3x3,red_5x5,out_5x5,out_pool_3x3):
        
        super(Inception_block,self).__init__()
        
        self.branch_1=nn.Sequential(Conv_block(in_channel,out_1x1,kernel_size=1))
        
        self.branch_2=nn.Sequential(
            Conv_block(in_channel,red_3x3,kernel_size=1),
            Conv_block(red_3x3,out_3x3,kernel_size=3,padding=1)
        )
        
        self.branch_3=nn.Sequential(
            Conv_block(in_channel,red_5x5,kernel_size=1),
            Conv_block(red_5x5,out_5x5,kernel_size=5,padding=2)
        )
        
        self.branch_4=nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            Conv_block(in_channel,out_pool_3x3,kernel_size=1)
        )
        
    def forward(self,x):
        return torch.cat(
            [self.branch_1(x),
             self.branch_2(x),
             self.branch_3(x),
             self.branch_4(x)],
            1
        )

In [4]:
class GoogleNet(nn.Module):
    def __init__(self,in_channel,out_channel):
        super(GoogleNet,self).__init__()
        
        self.conv_1=Conv_block(in_channel,64,kernel_size=7,padding=3,stride=2)
        
        self.pool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.conv_2=Conv_block(64,192,kernel_size=3,padding=1,stride=1)
        
        self.inception_3a=Inception_block(192,64,96,128,16,32,32)
        self.inception_3b=Inception_block(256,128,128,192,32,96,64)
        
        self.inception_4a=Inception_block(480,192,96,208,16,48,64)
        self.inception_4b=Inception_block(512,160,112,224,24,64,64)
        self.inception_4c=Inception_block(512,128,128,256,24,64,64)
        self.inception_4d=Inception_block(512,112,144,288,32,64,64)
        self.inception_4e=Inception_block(528,256,160,320,32,128,128)
        
        self.inception_5a=Inception_block(832,256,160,320,32,128,128)
        self.inception_5b=Inception_block(832,384,192,384,48,128,128)
        
        self.avgPool=nn.AvgPool2d(kernel_size=7,stride=1)
        
        self.drop=nn.Dropout(p=0.4)
        
        self.fc=nn.Linear(1024,out_channel)
        
        
    def forward(self,x):
        x=self.conv_1(x)
        
        x=self.pool(x)
        
        x=self.conv_2(x)
        
        x=self.pool(x)
        
        x=self.inception_3a(x)
        x=self.inception_3b(x)
        
        x=self.pool(x)
        
        x=self.inception_4a(x)
        x=self.inception_4b(x)
        x=self.inception_4c(x)
        x=self.inception_4d(x)
        x=self.inception_4e(x)
        
        x=self.pool(x)
        
        x=self.inception_5a(x)
        x=self.inception_5b(x)
        
        
        
        x=self.avgPool(x)
        
        x=x.reshape(x.shape[0],-1)

        
        x=self.drop(x)
        
        x=self.fc(x)
        
        return x

In [5]:
x=torch.rand(3,3,224,224)

In [6]:
model=GoogleNet(in_channel=3,out_channel=10)

In [7]:
model(x)[2].shape

torch.Size([10])

In [8]:
print(summary(model,x,verbose=0)
    

SyntaxError: unexpected EOF while parsing (4212240097.py, line 2)

In [9]:
model

GoogleNet(
  (conv_1): Conv_block(
    (network): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv_2): Conv_block(
    (network): Sequential(
      (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (inception_3a): Inception_block(
    (branch_1): Sequential(
      (0): Conv_block(
        (network): Sequential(
          (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
    )
    (branch_2): Sequential(
      (0): Conv_block(
        (network): Sequential(
   